In [1]:
import pandas as pd 
import numpy as np 
import scanpy as sc
import pycisTopic
pycisTopic.__version__

import os
out_dir = "scenic_brain_out"
os.makedirs(out_dir, exist_ok = True)

/home/rsun@ZHANGroup.local/anaconda3/envs/scenicplus/lib/python3.11/site-packages/pycisTopic/__init__.py:3: UserWarning: pkg_resources is deprecated as an API. See https://setuptools.pypa.io/en/latest/pkg_resources.html. The pkg_resources package is slated for removal as early as 2025-11-30. Refrain from using this package or pin to Setuptools<81.
  from pkg_resources import DistributionNotFound, get_distribution


# Description 

this notebook uses pycistopic to generate the cistopic object data required for the scenicplus analysis 

<font color='red'> 
Noted that due to the implementation of pycistopic and the dataset size of our sc-multiome data, running this notebook requires a large amount of memory and a very long time.
</font>

## get cell anno

In [3]:
"""
scdata = sc.read_h5ad('/home/rsun@ZHANGroup.local/cross_embed/data/species_raw.h5ad')
obs_info =scdata.obs.copy()
new_idx = []
for ele in scdata.obs.index:
    a,b,c = ele.split('-')
    key = b + '-' + c +'_' + a
    new_idx.append(key)
obs_info.index = new_idx 
obs_info.to_csv(os.path.join(out_dir,'cell_data.csv'), index = True)
"""

"\nscdata = sc.read_h5ad('/home/rsun@ZHANGroup.local/cross_embed/data/species_raw.h5ad')\nobs_info =scdata.obs.copy()\nnew_idx = []\nfor ele in scdata.obs.index:\n    a,b,c = ele.split('-')\n    key = b + '-' + c +'_' + a\n    new_idx.append(key)\nobs_info.index = new_idx \nobs_info.to_csv(os.path.join(out_dir,'cell_data.csv'), index = True)\n"

In [5]:
cell_data = pd.read_csv(os.path.join(out_dir, 'cell_data.csv'), index_col=0)
#cell_data

## get pseudo bulk

In [6]:
chromsizes = pd.read_table(
    "http://hgdownload.cse.ucsc.edu/goldenPath/mm10/bigZips/mm10.chrom.sizes",
    header = None,
    names = ["Chromosome", "End"]
)

chromsizes.insert(1, "Start", 0)
chromsizes.head()

fragments_dict = {}
root = '/home/rsun@ZHANGroup.local/sly_data/sly_07_exfig/global_grn/fragments'
for ele in os.listdir(root):
    sub_path = os.path.join(root, ele)
    frag_path = os.path.join(sub_path, 'fragment.tsv.gz')
    sample,_ = ele.split('_')
    fragments_dict[sample] = frag_path
print(len(fragments_dict))

40


In [11]:
from pycisTopic.pseudobulk_peak_calling import export_pseudobulk
os.makedirs(os.path.join(out_dir, "consensus_peak_calling"), exist_ok = True)
os.makedirs(os.path.join(out_dir, "consensus_peak_calling/pseudobulk_bed_files"), exist_ok = True)
os.makedirs(os.path.join(out_dir, "consensus_peak_calling/pseudobulk_bw_files"), exist_ok = True)
os.makedirs(os.path.join(out_dir, 'temp'), exist_ok = True)

bw_paths, bed_paths = export_pseudobulk(
    input_data = cell_data,
    variable = "tissues",
    sample_id_col = "samples",
    chromsizes = chromsizes,
    bed_path = os.path.join(out_dir, "consensus_peak_calling/pseudobulk_bed_files"),
    bigwig_path = os.path.join(out_dir, "consensus_peak_calling/pseudobulk_bw_files"),
    path_to_fragments = fragments_dict,
    n_cpu = 40,
    normalize_bigwig = True,
    temp_dir = os.path.join(out_dir, 'temp'),
    split_pattern = None #'_'
)

2025-07-25 17:05:40,931 cisTopic     INFO     Splitting fragments by cell type.
2025-07-25 17:11:26,446 cisTopic     INFO     generating bigwig files


/home/rsun@ZHANGroup.local/anaconda3/envs/scenicplus/lib/python3.11/site-packages/pycisTopic/__init__.py:3: UserWarning: pkg_resources is deprecated as an API. See https://setuptools.pypa.io/en/latest/pkg_resources.html. The pkg_resources package is slated for removal as early as 2025-11-30. Refrain from using this package or pin to Setuptools<81.
  from pkg_resources import DistributionNotFound, get_distribution
/home/rsun@ZHANGroup.local/anaconda3/envs/scenicplus/lib/python3.11/site-packages/pycisTopic/__init__.py:3: UserWarning: pkg_resources is deprecated as an API. See https://setuptools.pypa.io/en/latest/pkg_resources.html. The pkg_resources package is slated for removal as early as 2025-11-30. Refrain from using this package or pin to Setuptools<81.
  from pkg_resources import DistributionNotFound, get_distribution
/home/rsun@ZHANGroup.local/anaconda3/envs/scenicplus/lib/python3.11/site-packages/pycisTopic/__init__.py:3: UserWarning: pkg_resources is deprecated as an API. See ht

In [12]:
with open(os.path.join(out_dir, "consensus_peak_calling/bw_paths.tsv"), "wt") as f:
    for v in bw_paths:
        _ = f.write(f"{v}\t{bw_paths[v]}\n")

with open(os.path.join(out_dir, "consensus_peak_calling/bed_paths.tsv"), "wt") as f:
    for v in bed_paths:
        _ = f.write(f"{v}\t{bed_paths[v]}\n")

## infer consensus peaks

In [13]:
bw_paths = {}
with open(os.path.join(out_dir, "consensus_peak_calling/bw_paths.tsv")) as f:
    for line in f:
        v, p = line.strip().split("\t")
        bw_paths.update({v: p})

bed_paths = {}
with open(os.path.join(out_dir, "consensus_peak_calling/bed_paths.tsv")) as f:
    for line in f:
        v, p = line.strip().split("\t")
        bed_paths.update({v: p})

from pycisTopic.pseudobulk_peak_calling import peak_calling
macs_path = "/home/rsun@ZHANGroup.local/anaconda3/envs/pyscenic/bin/macs2"

os.makedirs(os.path.join(out_dir, "consensus_peak_calling/MACS"), exist_ok = True)

narrow_peak_dict = peak_calling(
    macs_path = macs_path,
    bed_paths = bed_paths,
    outdir = os.path.join(os.path.join(out_dir, "consensus_peak_calling/MACS")),
    genome_size = 'hs',
    n_cpu = 40,
    input_format = 'BEDPE',
    shift = 73,
    ext_size = 146,
    keep_dup = 'all',
    q_value = 0.05,
    _temp_dir = None
)

2025-07-25 17:14:55,056	INFO worker.py:1724 -- Started a local Ray instance.
(pid=3367357) /home/rsun@ZHANGroup.local/anaconda3/envs/scenicplus/lib/python3.11/site-packages/pycisTopic/__init__.py:3: UserWarning: pkg_resources is deprecated as an API. See https://setuptools.pypa.io/en/latest/pkg_resources.html. The pkg_resources package is slated for removal as early as 2025-11-30. Refrain from using this package or pin to Setuptools<81.
(pid=3367357)   from pkg_resources import DistributionNotFound, get_distribution


(macs_call_peak_ray pid=3367357) 2025-07-25 17:14:57,167 cisTopic     INFO     Calling peaks for NG with /home/rsun@ZHANGroup.local/anaconda3/envs/pyscenic/bin/macs2 callpeak --treatment scenic_brain_out/consensus_peak_calling/pseudobulk_bed_files/NG.fragments.tsv.gz --name NG  --outdir scenic_brain_out/consensus_peak_calling/MACS --format BEDPE --gsize hs --qvalue 0.05 --nomodel --shift 73 --extsize 146 --keep-dup all --call-summits --nolambda
(macs_call_peak_ray pid=3367346) 2025-07-25 17:14:57,179 cisTopic     INFO     Calling peaks for CT with /home/rsun@ZHANGroup.local/anaconda3/envs/pyscenic/bin/macs2 callpeak --treatment scenic_brain_out/consensus_peak_calling/pseudobulk_bed_files/CT.fragments.tsv.gz --name CT  --outdir scenic_brain_out/consensus_peak_calling/MACS --format BEDPE --gsize hs --qvalue 0.05 --nomodel --shift 73 --extsize 146 --keep-dup all --call-summits --nolambda
(macs_call_peak_ray pid=3367356) 2025-07-25 17:14:57,300 cisTopic     INFO     Calling peaks for QN wi

(pid=3367346) /home/rsun@ZHANGroup.local/anaconda3/envs/scenicplus/lib/python3.11/site-packages/pycisTopic/__init__.py:3: UserWarning: pkg_resources is deprecated as an API. See https://setuptools.pypa.io/en/latest/pkg_resources.html. The pkg_resources package is slated for removal as early as 2025-11-30. Refrain from using this package or pin to Setuptools<81. [repeated 7x across cluster] (Ray deduplicates logs by default. Set RAY_DEDUP_LOGS=0 to disable log deduplication, or see https://docs.ray.io/en/master/ray-observability/ray-logging.html#log-deduplication for more options.)
(pid=3367346)   from pkg_resources import DistributionNotFound, get_distribution [repeated 7x across cluster]


In [14]:
from pycisTopic.iterative_peak_calling import get_consensus_peaks
# Other param
peak_half_width=250
path_to_blacklist="/home/rsun@ZHANGroup.local/projects_list/pycisTopic-main/blacklist/mm10-blacklist.v2.bed"
# Get consensus peaks
consensus_peaks = get_consensus_peaks(
    narrow_peaks_dict = narrow_peak_dict,
    peak_half_width = peak_half_width,
    chromsizes = chromsizes,
    path_to_blacklist = path_to_blacklist)

consensus_peaks.to_bed(
    path = os.path.join(out_dir, "consensus_peak_calling/consensus_regions.bed"),
    keep =True,
    compression = 'infer',
    chain = False)

2025-07-25 17:25:22,560 cisTopic     INFO     Extending and merging peaks per class
2025-07-25 17:25:46,022 cisTopic     INFO     Normalizing peak scores
2025-07-25 17:25:46,557 cisTopic     INFO     Merging peaks
Warning! Start and End columns now have different dtypes: int64 and int32
2025-07-25 17:27:12,788 cisTopic     INFO     Done!


/home/rsun@ZHANGroup.local/anaconda3/envs/scenicplus/lib/python3.11/site-packages/pyranges/out.py:37: FutureWarning: Passing a set as an indexer is deprecated and will raise in a future version. Use a list instead.
  return pd.concat([outdf, df.get(noncanonical)], axis=1)
/home/rsun@ZHANGroup.local/anaconda3/envs/scenicplus/lib/python3.11/site-packages/pyranges/out.py:37: FutureWarning: Passing a set as an indexer is deprecated and will raise in a future version. Use a list instead.
  return pd.concat([outdf, df.get(noncanonical)], axis=1)
/home/rsun@ZHANGroup.local/anaconda3/envs/scenicplus/lib/python3.11/site-packages/pyranges/out.py:37: FutureWarning: Passing a set as an indexer is deprecated and will raise in a future version. Use a list instead.
  return pd.concat([outdf, df.get(noncanonical)], axis=1)
/home/rsun@ZHANGroup.local/anaconda3/envs/scenicplus/lib/python3.11/site-packages/pyranges/out.py:37: FutureWarning: Passing a set as an indexer is deprecated and will raise in a fu

## QC

In [15]:
!pycistopic tss gene_annotation_list | grep Mouse
!mkdir -p scenic_brain_out/qc
!pycistopic tss get_tss \
    --output scenic_brain_out/qc/tss.bed \
    --name "mmusculus_gene_ensembl" \
    --to-chrom-source ucsc \
    --ucsc mm10
!head scenic_brain_out/qc/tss.bed | column -t

/home/rsun@ZHANGroup.local/anaconda3/envs/scenicplus/lib/python3.11/site-packages/pycisTopic/__init__.py:3: UserWarning: pkg_resources is deprecated as an API. See https://setuptools.pypa.io/en/latest/pkg_resources.html. The pkg_resources package is slated for removal as early as 2025-11-30. Refrain from using this package or pin to Setuptools<81.
  from pkg_resources import DistributionNotFound, get_distribution
mmurinus_gene_ensembl	Mouse Lemur genes (Mmur_3.0)
mmusculus_gene_ensembl	Mouse genes (GRCm39)
/home/rsun@ZHANGroup.local/anaconda3/envs/scenicplus/lib/python3.11/site-packages/pycisTopic/__init__.py:3: UserWarning: pkg_resources is deprecated as an API. See https://setuptools.pypa.io/en/latest/pkg_resources.html. The pkg_resources package is slated for removal as early as 2025-11-30. Refrain from using this package or pin to Setuptools<81.
  from pkg_resources import DistributionNotFound, get_distribution
- Get TSS annotation from Ensembl BioMart with the following settings:


In [16]:
## calculate qc metrics parallel 

regions_bed_filename = os.path.join(out_dir, "consensus_peak_calling/consensus_regions.bed")
tss_bed_filename = os.path.join(out_dir, "qc", "tss.bed")

pycistopic_qc_commands_filename = "pycistopic_qc_commands.txt"

# Create text file with all pycistopic qc command lines.
with open(pycistopic_qc_commands_filename, "w") as fh:
    for sample, fragment_filename in fragments_dict.items():
        print(
            "pycistopic qc",
            f"--fragments {fragment_filename}",
            f"--regions {regions_bed_filename}",
            f"--tss {tss_bed_filename}",
            f"--output {os.path.join(out_dir, 'qc')}/{sample}",
            sep=" ",
            file=fh,
        )

!cat pycistopic_qc_commands.txt | parallel -j 40 {}

Academic tradition requires you to cite works you base your article on.
If you use programs that use GNU Parallel to process data for an article in a
scientific publication, please cite:

  Tange, O. (2025, April 22). GNU Parallel 20250422 ('Tariffs').
  Zenodo. https://doi.org/10.5281/zenodo.15265748

This helps funding further development; AND IT WON'T COST YOU A CENT.
If you pay 10000 EUR you should feel free to use GNU Parallel without citing.

More about funding GNU Parallel and the citation notice:
https://www.gnu.org/software/parallel/parallel_design.html#citation-notice

To silence this citation notice: run 'parallel --citation' once.

Come on: You have run parallel 10 times. Isn't it about time 
you run 'parallel --citation' once to silence the citation notice?

/home/rsun@ZHANGroup.local/anaconda3/envs/scenicplus/lib/python3.11/site-packages/pycisTopic/__init__.py:3: UserWarning: pkg_resources is deprecated as an API. See https://setuptools.pypa.io/en/latest/pkg_resources.htm

### qcplot

In [17]:
from pycisTopic.plotting.qc_plot import plot_sample_stats, plot_barcode_stats
import matplotlib.pyplot as plt

#for sample_id in fragments_dict:
#    fig = plot_sample_stats(
#        sample_id = sample_id,
#        pycistopic_qc_output_dir = "scenic_brain_out/qc"
#    )
## 绘图 ## 全部清除，控制.ipynb大小

In [18]:
from pycisTopic.qc import get_barcodes_passing_qc_for_sample
sample_id_to_barcodes_passing_filters = {}
sample_id_to_thresholds = {}

for sample_id in fragments_dict:
    (
        sample_id_to_barcodes_passing_filters[sample_id],
        sample_id_to_thresholds[sample_id]
    ) = get_barcodes_passing_qc_for_sample(
            sample_id = sample_id,
            pycistopic_qc_output_dir = os.path.join(out_dir,'qc'),
            unique_fragments_threshold = 500, # use automatic thresholding
            tss_enrichment_threshold = 0.5, # use automatic thresholding
            frip_threshold = 0,
            use_automatic_thresholds = True,
    )
"""
for sample_id in fragments_dict:
    fig = plot_barcode_stats(
        sample_id = sample_id,
        pycistopic_qc_output_dir = os.path.join(out_dir,'qc'),
        bc_passing_filters = sample_id_to_barcodes_passing_filters[sample_id],
        detailed_title = False,
        **sample_id_to_thresholds[sample_id]
    )
"""
## TSS 分数很低

CYYB:
	Using user-defined threshold for unique fragments: 500
	Using user-defined threshold for TSS enrichment: 0.5
MCT:
	Using user-defined threshold for unique fragments: 500
	Using user-defined threshold for TSS enrichment: 0.5
CYXQ:
	Using user-defined threshold for unique fragments: 500
	Using user-defined threshold for TSS enrichment: 0.5
JTNG:
	Using user-defined threshold for unique fragments: 500
	Using user-defined threshold for TSS enrichment: 0.5
CYQN:
	Using user-defined threshold for unique fragments: 500
	Using user-defined threshold for TSS enrichment: 0.5
QFYB:
	Using user-defined threshold for unique fragments: 500
	Using user-defined threshold for TSS enrichment: 0.5
QFNG:
	Using user-defined threshold for unique fragments: 500
	Using user-defined threshold for TSS enrichment: 0.5
QFQN:
	Using user-defined threshold for unique fragments: 500
	Using user-defined threshold for TSS enrichment: 0.5
JTYB:
	Using user-defined threshold for unique fragments: 500
	Using user

"\nfor sample_id in fragments_dict:\n    fig = plot_barcode_stats(\n        sample_id = sample_id,\n        pycistopic_qc_output_dir = os.path.join(out_dir,'qc'),\n        bc_passing_filters = sample_id_to_barcodes_passing_filters[sample_id],\n        detailed_title = False,\n        **sample_id_to_thresholds[sample_id]\n    )\n"

## create cistopic object

In [19]:
path_to_regions = os.path.join(out_dir, "consensus_peak_calling/consensus_regions.bed")
#path_to_blacklist = "pycisTopic/blacklist/hg38-blacklist.v2.bed"
path_to_blacklist="/home/rsun@ZHANGroup.local/projects_list/pycisTopic-main/blacklist/mm10-blacklist.v2.bed"
pycistopic_qc_output_dir = os.path.join(out_dir,'qc')

from pycisTopic.cistopic_class import create_cistopic_object_from_fragments
import polars as pl

cistopic_obj_list = []
for sample_id in fragments_dict:
    sample_metrics = pl.read_parquet(
        os.path.join(pycistopic_qc_output_dir, f'{sample_id}.fragments_stats_per_cb.parquet')
    ).to_pandas().set_index("CB").loc[ sample_id_to_barcodes_passing_filters[sample_id] ]

    """process cell id"""
    #new_idx = []
    #for ele in sample_metrics.index.values:
    #    new_idx.append(ele +'--')
    #sample_metrics.index = new_idx
    
    cistopic_obj = create_cistopic_object_from_fragments(
        path_to_fragments = fragments_dict[sample_id],
        path_to_regions = path_to_regions,
        path_to_blacklist = path_to_blacklist,
        metrics = sample_metrics,
        valid_bc = sample_id_to_barcodes_passing_filters[sample_id],
        n_cpu = 10,
        project = sample_id,
        split_pattern = '_'
    )
    cistopic_obj_list.append(cistopic_obj)

2025-07-25 17:30:02,617 cisTopic     INFO     Reading data for CYYB


2025-07-25 17:30:29,456 cisTopic     INFO     metrics provided!
2025-07-25 17:30:31,048 cisTopic     INFO     Counting fragments in regions


2025-07-25 17:30:36,625	INFO worker.py:1724 -- Started a local Ray instance.
(pid=3404799) /home/rsun@ZHANGroup.local/anaconda3/envs/scenicplus/lib/python3.11/site-packages/pyranges/__init__.py:18: UserWarning: pkg_resources is deprecated as an API. See https://setuptools.pypa.io/en/latest/pkg_resources.html. The pkg_resources package is slated for removal as early as 2025-11-30. Refrain from using this package or pin to Setuptools<81.
(pid=3404799)   import pkg_resources
(pid=3404794) /home/rsun@ZHANGroup.local/anaconda3/envs/scenicplus/lib/python3.11/site-packages/pyranges/__init__.py:18: UserWarning: pkg_resources is deprecated as an API. See https://setuptools.pypa.io/en/latest/pkg_resources.html. The pkg_resources package is slated for removal as early as 2025-11-30. Refrain from using this package or pin to Setuptools<81. [repeated 9x across cluster]
(pid=3404794)   import pkg_resources [repeated 9x across cluster]


2025-07-25 17:30:46,716 cisTopic     INFO     Creating fragment matrix
2025-07-25 17:31:08,497 cisTopic     INFO     Converting fragment matrix to sparse matrix
2025-07-25 17:31:11,072 cisTopic     INFO     Removing blacklisted regions
2025-07-25 17:31:11,430 cisTopic     INFO     Creating CistopicObject
2025-07-25 17:31:12,210 cisTopic     INFO     Done!
2025-07-25 17:31:12,283 cisTopic     INFO     Reading data for MCT
2025-07-25 17:31:46,421 cisTopic     INFO     metrics provided!
2025-07-25 17:31:48,480 cisTopic     INFO     Counting fragments in regions


2025-07-25 17:31:55,261	INFO worker.py:1724 -- Started a local Ray instance.
(pid=3407704) /home/rsun@ZHANGroup.local/anaconda3/envs/scenicplus/lib/python3.11/site-packages/pyranges/__init__.py:18: UserWarning: pkg_resources is deprecated as an API. See https://setuptools.pypa.io/en/latest/pkg_resources.html. The pkg_resources package is slated for removal as early as 2025-11-30. Refrain from using this package or pin to Setuptools<81.
(pid=3407704)   import pkg_resources
(pid=3407707) /home/rsun@ZHANGroup.local/anaconda3/envs/scenicplus/lib/python3.11/site-packages/pyranges/__init__.py:18: UserWarning: pkg_resources is deprecated as an API. See https://setuptools.pypa.io/en/latest/pkg_resources.html. The pkg_resources package is slated for removal as early as 2025-11-30. Refrain from using this package or pin to Setuptools<81. [repeated 9x across cluster]
(pid=3407707)   import pkg_resources [repeated 9x across cluster]


...

2025-07-25 18:21:17,519 cisTopic     INFO     Creating fragment matrix
2025-07-25 18:21:35,346 cisTopic     INFO     Converting fragment matrix to sparse matrix
2025-07-25 18:21:38,387 cisTopic     INFO     Removing blacklisted regions
2025-07-25 18:21:38,665 cisTopic     INFO     Creating CistopicObject
2025-07-25 18:21:39,215 cisTopic     INFO     Done!
2025-07-25 18:21:39,281 cisTopic     INFO     Reading data for TXQ
2025-07-25 18:22:13,392 cisTopic     INFO     metrics provided!
2025-07-25 18:22:14,630 cisTopic     INFO     Counting fragments in regions


2025-07-25 18:22:23,706	INFO worker.py:1724 -- Started a local Ray instance.
(pid=3517558) /home/rsun@ZHANGroup.local/anaconda3/envs/scenicplus/lib/python3.11/site-packages/pyranges/__init__.py:18: UserWarning: pkg_resources is deprecated as an API. See https://setuptools.pypa.io/en/latest/pkg_resources.html. The pkg_resources package is slated for removal as early as 2025-11-30. Refrain from using this package or pin to Setuptools<81.
(pid=3517558)   import pkg_resources
(pid=3517552) /home/rsun@ZHANGroup.local/anaconda3/envs/scenicplus/lib/python3.11/site-packages/pyranges/__init__.py:18: UserWarning: pkg_resources is deprecated as an API. See https://setuptools.pypa.io/en/latest/pkg_resources.html. The pkg_resources package is slated for removal as early as 2025-11-30. Refrain from using this package or pin to Setuptools<81. [repeated 9x across cluster]
(pid=3517552)   import pkg_resources [repeated 9x across cluster]


2025-07-25 18:22:34,336 cisTopic     INFO     Creating fragment matrix
2025-07-25 18:23:00,306 cisTopic     INFO     Converting fragment matrix to sparse matrix
2025-07-25 18:23:05,126 cisTopic     INFO     Removing blacklisted regions
2025-07-25 18:23:05,432 cisTopic     INFO     Creating CistopicObject
2025-07-25 18:23:06,141 cisTopic     INFO     Done!
2025-07-25 18:23:06,222 cisTopic     INFO     Reading data for TCT
2025-07-25 18:23:31,760 cisTopic     INFO     metrics provided!
2025-07-25 18:23:32,836 cisTopic     INFO     Counting fragments in regions


2025-07-25 18:23:40,897	INFO worker.py:1724 -- Started a local Ray instance.
(pid=3520553) /home/rsun@ZHANGroup.local/anaconda3/envs/scenicplus/lib/python3.11/site-packages/pyranges/__init__.py:18: UserWarning: pkg_resources is deprecated as an API. See https://setuptools.pypa.io/en/latest/pkg_resources.html. The pkg_resources package is slated for removal as early as 2025-11-30. Refrain from using this package or pin to Setuptools<81.
(pid=3520553)   import pkg_resources
(pid=3520546) /home/rsun@ZHANGroup.local/anaconda3/envs/scenicplus/lib/python3.11/site-packages/pyranges/__init__.py:18: UserWarning: pkg_resources is deprecated as an API. See https://setuptools.pypa.io/en/latest/pkg_resources.html. The pkg_resources package is slated for removal as early as 2025-11-30. Refrain from using this package or pin to Setuptools<81. [repeated 9x across cluster]
(pid=3520546)   import pkg_resources [repeated 9x across cluster]


2025-07-25 18:23:49,907 cisTopic     INFO     Creating fragment matrix
2025-07-25 18:24:14,031 cisTopic     INFO     Converting fragment matrix to sparse matrix
2025-07-25 18:24:17,845 cisTopic     INFO     Removing blacklisted regions
2025-07-25 18:24:18,126 cisTopic     INFO     Creating CistopicObject
2025-07-25 18:24:18,775 cisTopic     INFO     Done!


In [20]:
from pycisTopic.cistopic_class import merge 

cistopic_obj = merge(cistopic_obj_list, split_pattern = '_')
print(cistopic_obj)

2025-07-26 03:44:29,939 cisTopic     INFO     cisTopic object 1 merged
2025-07-26 03:44:42,493 cisTopic     INFO     cisTopic object 2 merged
2025-07-26 03:44:57,748 cisTopic     INFO     cisTopic object 3 merged
2025-07-26 03:45:23,799 cisTopic     INFO     cisTopic object 4 merged
2025-07-26 03:45:45,421 cisTopic     INFO     cisTopic object 5 merged
2025-07-26 03:46:06,667 cisTopic     INFO     cisTopic object 6 merged
2025-07-26 03:46:42,275 cisTopic     INFO     cisTopic object 7 merged
2025-07-26 03:47:03,245 cisTopic     INFO     cisTopic object 8 merged
2025-07-26 03:47:48,541 cisTopic     INFO     cisTopic object 9 merged
2025-07-26 03:48:16,025 cisTopic     INFO     cisTopic object 10 merged
2025-07-26 03:49:30,560 cisTopic     INFO     cisTopic object 11 merged
2025-07-26 03:49:54,386 cisTopic     INFO     cisTopic object 12 merged
2025-07-26 03:50:14,152 cisTopic     INFO     cisTopic object 13 merged
2025-07-26 03:52:01,711 cisTopic     INFO     cisTopic object 14 merged
2

In [24]:
from pycisTopic.cistopic_class import merge 

#cistopic_obj = merge(cistopic_obj_list, split_pattern = '-')
#print(cistopic_obj)

import pickle
pickle.dump(
    cistopic_obj,
    open(os.path.join(out_dir, "cistopic_obj.pkl"), "wb")
)

cistopic_obj.add_cell_data(cell_data, split_pattern='_')
pickle.dump(
    cistopic_obj,
    open(os.path.join(out_dir, "cistopic_obj.pkl"), "wb")
)

## add metadata

In [27]:
cistopic_obj.add_cell_data(cell_data, split_pattern='_')
#cistopic_obj.cell_data
pickle.dump(
    cistopic_obj,
    open(os.path.join(out_dir, "cistopic_obj.pkl"), "wb")
)

Columns ['tissues', 'samples', 'total_counts', 'n_genes_by_counts', 'coarse_celltype', 'pct_counts_mt', 'n_genes', 'species', 'total_counts_mt', 'cluster', 'leiden', 'celltype'] will be overwritten


## LDA model

In [1]:
lda_path = os.path.join(out_dir, 'lda_models')
os.makedirs(lda_path, exist_ok=True)
os.environ['MALLET_MEMORY'] = '200G'
from pycisTopic.lda_models import run_cgs_models_mallet
# Configure path Mallet
mallet_path="/home/rsun@ZHANGroup.local/projects_list/Mallet-202108/bin/mallet"
# Run models
models=run_cgs_models_mallet(
    cistopic_obj,
    n_topics=[30, 40],
    n_cpu=128,
    n_iter=500,
    random_state=555,
    alpha=50,
    alpha_by_topic=True,
    eta=0.1,
    eta_by_topic=False,
    tmp_path=lda_path,
    save_path=lda_path,
    mallet_path=mallet_path,
)

pickle.dump(
    models,
    open(os.path.join(lda_path, "models.pkl"), "wb")
)

NameError: name 'os' is not defined

## model selection

In [1]:
from pycisTopic.lda_models import evaluate_models
model = evaluate_models(
    models,
    select_model = 40,
    return_model = True
)

cistopic_obj.add_LDA_model(model)
pickle.dump(
    cistopic_obj,
    open(os.path.join(out_dir, "cistopic_obj.pkl"), "wb")
)

/home/rsun@ZHANGroup.local/anaconda3/envs/scenicplus/lib/python3.11/site-packages/pycisTopic/__init__.py:3: UserWarning: pkg_resources is deprecated as an API. See https://setuptools.pypa.io/en/latest/pkg_resources.html. The pkg_resources package is slated for removal as early as 2025-11-30. Refrain from using this package or pin to Setuptools<81.
  from pkg_resources import DistributionNotFound, get_distribution
/home/rsun@ZHANGroup.local/anaconda3/envs/scenicplus/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
2025-07-28 13:41:18,361	INFO util.py:154 -- Missing packages: ['ipywidgets']. Run `pip install -U ipywidgets`, then restart the notebook server for rich notebook output.


NameError: name 'models' is not defined

## 重新加载cistopic 

In [4]:


import pickle
import pandas as pd 
import numpy as np 
import scanpy as sc
import pycisTopic
pycisTopic.__version__

import os
out_dir = "scenic_brain_out"
os.makedirs(out_dir, exist_ok = True)

file_path = os.path.join(out_dir, "cistopic_obj.pkl")

with open(file_path, 'rb') as file:
    cistopic_obj = pickle.load(file)

"""
lda_path = os.path.join(out_dir, 'lda_models')
file_path = os.path.join(lda_path, "models.pkl")
with open(file_path, 'rb') as file:
    models = pickle.load(file)

from pycisTopic.lda_models import evaluate_models
model = evaluate_models(
    models,
    select_model = 40,
    return_model = True
)
"""


'\nlda_path = os.path.join(out_dir, \'lda_models\')\nfile_path = os.path.join(lda_path, "models.pkl")\nwith open(file_path, \'rb\') as file:\n    models = pickle.load(file)\n\nfrom pycisTopic.lda_models import evaluate_models\nmodel = evaluate_models(\n    models,\n    select_model = 40,\n    return_model = True\n)\n'

## cluster and vis

In [ ]:
fig_dir = os.path.join(out_dir, 'figures')
os.makedirs(fig_dir, exist_ok=True)

from pycisTopic.clust_vis import (
    find_clusters,
    run_umap,
    run_tsne,
    plot_metadata,
    plot_topic,
    cell_topic_heatmap
)

"""
find_clusters(
    cistopic_obj,
    target  = 'cell',
    k = 10,
    res = [0.25, 0.5],
    prefix = 'pycisTopic_',
    scale = True,
    split_pattern = '_'
)
"""
run_umap(
    cistopic_obj,
    target  = 'cell', scale=True)



plot_metadata(
    cistopic_obj,
    reduction_name='UMAP',
    variables=['species','tissues','celltype','coarse_celltype'],
    target='cell',
    text_size=10,
    dot_size=5,
    save = os.path.join(fig_dir,'topic_umap.pdf'))

In [ ]:
cell_topic_heatmap(
    cistopic_obj,
    variables = ['tissues'],
    scale = False,
    legend_loc_x = 1.0,
    legend_loc_y = -1.2,
    legend_dist_y = -1,
    figsize = (10, 10)
)

## TOPIC binary + QC

In [5]:
from pycisTopic.topic_binarization import binarize_topics

region_bin_topics_top_3k = binarize_topics(
    cistopic_obj, method='ntop', ntop = 3_000,
    plot=False, num_columns=5
)

In [6]:
region_bin_topics_otsu = binarize_topics(
    cistopic_obj, method='otsu',
    plot=False, num_columns=5
)

In [7]:
binarized_cell_topic = binarize_topics(
    cistopic_obj,
    target='cell',
    method='li',
    plot=False,
    num_columns=5, nbins=100)

In [15]:
from pycisTopic.topic_qc import compute_topic_metrics, plot_topic_qc, topic_annotation
import matplotlib.pyplot as plt
from pycisTopic.utils import fig2img

topic_qc_metrics = compute_topic_metrics(cistopic_obj)
"""
fig_dict={}
fig_dict['CoherenceVSAssignments']=plot_topic_qc(topic_qc_metrics, var_x='Coherence', var_y='Log10_Assignments', var_color='Gini_index', plot=False, return_fig=True)
fig_dict['AssignmentsVSCells_in_bin']=plot_topic_qc(topic_qc_metrics, var_x='Log10_Assignments', var_y='Cells_in_binarized_topic', var_color='Gini_index', plot=False, return_fig=True)
fig_dict['CoherenceVSCells_in_bin']=plot_topic_qc(topic_qc_metrics, var_x='Coherence', var_y='Cells_in_binarized_topic', var_color='Gini_index', plot=False, return_fig=True)
fig_dict['CoherenceVSRegions_in_bin']=plot_topic_qc(topic_qc_metrics, var_x='Coherence', var_y='Regions_in_binarized_topic', var_color='Gini_index', plot=False, return_fig=True)
fig_dict['CoherenceVSMarginal_dist']=plot_topic_qc(topic_qc_metrics, var_x='Coherence', var_y='Marginal_topic_dist', var_color='Gini_index', plot=False, return_fig=True)
fig_dict['CoherenceVSGini_index']=plot_topic_qc(topic_qc_metrics, var_x='Coherence', var_y='Gini_index', var_color='Gini_index', plot=False, return_fig=True)

# Plot topic stats in one figure
fig=plt.figure(figsize=(40, 43))
i = 1
for fig_ in fig_dict.keys():
    plt.subplot(2, 3, i)
    img = fig2img(fig_dict[fig_]) #To convert figures to png to plot together, see .utils.py. This converts the figure to png.
    plt.imshow(img)
    plt.axis('off')
    i += 1
plt.subplots_adjust(wspace=0, hspace=-0.70)
plt.show()
"""

"\nfig_dict={}\nfig_dict['CoherenceVSAssignments']=plot_topic_qc(topic_qc_metrics, var_x='Coherence', var_y='Log10_Assignments', var_color='Gini_index', plot=False, return_fig=True)\nfig_dict['AssignmentsVSCells_in_bin']=plot_topic_qc(topic_qc_metrics, var_x='Log10_Assignments', var_y='Cells_in_binarized_topic', var_color='Gini_index', plot=False, return_fig=True)\nfig_dict['CoherenceVSCells_in_bin']=plot_topic_qc(topic_qc_metrics, var_x='Coherence', var_y='Cells_in_binarized_topic', var_color='Gini_index', plot=False, return_fig=True)\nfig_dict['CoherenceVSRegions_in_bin']=plot_topic_qc(topic_qc_metrics, var_x='Coherence', var_y='Regions_in_binarized_topic', var_color='Gini_index', plot=False, return_fig=True)\nfig_dict['CoherenceVSMarginal_dist']=plot_topic_qc(topic_qc_metrics, var_x='Coherence', var_y='Marginal_topic_dist', var_color='Gini_index', plot=False, return_fig=True)\nfig_dict['CoherenceVSGini_index']=plot_topic_qc(topic_qc_metrics, var_x='Coherence', var_y='Gini_index', va

In [ ]:
topic_annot = topic_annotation(
    cistopic_obj,
    annot_var='tissues',
    binarized_cell_topic=binarized_cell_topic,
    general_topic_thr = 0.2
)
topic_annot

/home/rsun@ZHANGroup.local/anaconda3/envs/scenicplus/lib/python3.11/site-packages/statsmodels/stats/weightstats.py:792: RuntimeWarning: divide by zero encountered in scalar divide
  zstat = value / std


,tissues,Ratio_cells_in_topic,Ratio_group_in_population,is_general
Topic1,"QN, NG",0.049167,0.244846,False
Topic2,"PC, QN, NG",0.063913,0.352485,False
Topic3,PC,0.022593,0.107639,False
Topic4,"BQ, XQ, YB",0.333248,0.386873,False
Topic5,"NG, CT",0.037491,0.271937,False
Topic6,"BQ, YB",0.063142,0.225518,False
Topic7,XQ,0.03092,0.161355,False
Topic8,CT,0.020934,0.150939,False
Topic9,"PC, HM",0.044483,0.217342,False
Topic10,"HM, QN, NG",0.047744,0.35455,False


: 

## DARs

In [8]:
from pycisTopic.diff_features import (
    impute_accessibility,
    normalize_scores,
    find_highly_variable_features,
    find_diff_features
)
import numpy as np

imputed_acc_obj = impute_accessibility(
    cistopic_obj,
    selected_cells=None,
    selected_regions=None,
    scale_factor=10**6
)


2025-07-28 13:42:54,276 cisTopic     INFO     Imputing region accessibility
2025-07-28 13:42:54,277 cisTopic     INFO     Impute region accessibility for regions 0-20000
2025-07-28 13:43:14,365 cisTopic     INFO     Impute region accessibility for regions 20000-40000
2025-07-28 13:43:33,431 cisTopic     INFO     Impute region accessibility for regions 40000-60000
2025-07-28 13:43:56,656 cisTopic     INFO     Impute region accessibility for regions 60000-80000
2025-07-28 13:44:21,398 cisTopic     INFO     Impute region accessibility for regions 80000-100000
2025-07-28 13:44:41,802 cisTopic     INFO     Impute region accessibility for regions 100000-120000
2025-07-28 13:45:01,943 cisTopic     INFO     Impute region accessibility for regions 120000-140000
2025-07-28 13:45:22,696 cisTopic     INFO     Impute region accessibility for regions 140000-160000
2025-07-28 13:45:55,535 cisTopic     INFO     Impute region accessibility for regions 160000-180000
2025-07-28 13:46:14,631 cisTopic     

In [9]:
## 原始的 cistopic的 normalize太慢了。。。

from tqdm import tqdm 
X = imputed_acc_obj.mtx
X = X.astype(np.float32)
b = 100
scale_factor = 1e6

result = []
for i in tqdm(range(0,X.shape[1],b)):
    tmp_x = X[:, i:min(i+b, X.shape[1])]
    scale_x = np.log1p(scale_factor * (tmp_x / (np.sum(tmp_x, axis = 0))))
    X[:,i:min(i+b, X.shape[1])] = scale_x    

100%|██████████| 2959/2959 [26:26<00:00,  1.86it/s]


In [10]:
from pycisTopic.cistopic_class import CistopicObject
from pycisTopic.diff_features import CistopicImputedFeatures
normalized_imputed_acc_obj =    CistopicImputedFeatures(
            X,
            imputed_acc_obj.feature_names,
            imputed_acc_obj.cell_names,
            imputed_acc_obj.project,
        )

In [11]:
#normalized_imputed_acc_obj = normalize_scores(imputed_acc_obj, scale_factor=10**4)

variable_regions = find_highly_variable_features(
    normalized_imputed_acc_obj,
    min_disp = 0.05,
    min_mean = 0.0125,
    max_mean = 3,
    max_disp = np.inf,
    n_bins=20,
    #n_top_features=6000,
    plot=False
)

2025-07-28 14:17:58,729 cisTopic     INFO     Calculating mean
2025-07-28 14:18:30,875 cisTopic     INFO     Calculating variance
2025-07-28 14:21:25,632 cisTopic     INFO     Done!


<Figure size 640x480 with 0 Axes>

In [12]:
print(len(variable_regions))

markers_dict= find_diff_features(
    cistopic_obj,
    normalized_imputed_acc_obj,  # ori imputed_acc_obj, but imputed_acc_obj is modified, so just use normalized_imputed_acc_obj
    #imputed_acc_obj,
    variable='tissues',
    var_features=variable_regions,
    contrasts=None,
    adjpval_thr=0.05,
    log2fc_thr=np.log2(1.5),
    n_cpu=40,
    _temp_dir= None, #os.path.join(out_dir, 'dar_temp'),
    split_pattern = '_'
)

"""
from pycisTopic.clust_vis import plot_imputed_features
plot_imputed_features(
    cistopic_obj,
    reduction_name='UMAP',
    imputed_data=imputed_acc_obj,
    features=[markers_dict[x].index.tolist()[0] for x in ['1','2','4','6']],
    scale=False,
    num_columns=4
)

print("Number of DARs found:")
print("---------------------")
for x in markers_dict:
    print(f"  {x}: {len(markers_dict[x])}")
"""

181246


2025-07-28 14:23:30,217	INFO worker.py:1724 -- Started a local Ray instance.


2025-07-28 14:23:31,046 cisTopic     INFO     Subsetting data for BQ (33637 of 295888)
2025-07-28 14:23:52,407 cisTopic     INFO     Computing p-value for BQ


(raylet) Spilled 23256 MiB, 1 objects, write throughput 252 MiB/s. Set RAY_verbose_spill_logs=0 to disable this message.
(raylet) Spilled 204576 MiB, 2 objects, write throughput 233 MiB/s.
(get_wilcox_test_pvalues_ray pid=960541) /home/rsun@ZHANGroup.local/anaconda3/envs/scenicplus/lib/python3.11/site-packages/ray/_private/pydantic_compat.py:2: UserWarning: pkg_resources is deprecated as an API. See https://setuptools.pypa.io/en/latest/pkg_resources.html. The pkg_resources package is slated for removal as early as 2025-11-30. Refrain from using this package or pin to Setuptools<81.
(get_wilcox_test_pvalues_ray pid=960541)   from pkg_resources import packaging


2025-07-28 14:46:09,662 cisTopic     INFO     Computing log2FC for BQ
2025-07-28 14:46:17,211 cisTopic     INFO     BQ done!
2025-07-28 14:46:40,223 cisTopic     INFO     Subsetting data for CT (44661 of 295888)
2025-07-28 14:46:52,922 cisTopic     INFO     Computing p-value for CT


...

(raylet) Spilled 840326 MiB, 9 objects, write throughput 236 MiB/s.


2025-07-28 16:12:57,930 cisTopic     INFO     Computing log2FC for PC
2025-07-28 16:12:59,236 cisTopic     INFO     PC done!
2025-07-28 16:13:26,533 cisTopic     INFO     Subsetting data for QN (36645 of 295888)
2025-07-28 16:13:39,046 cisTopic     INFO     Computing p-value for QN


(raylet) Spilled 1227459 MiB, 12 objects, write throughput 237 MiB/s.


2025-07-28 16:33:41,041 cisTopic     INFO     Computing log2FC for QN
2025-07-28 16:33:42,365 cisTopic     INFO     QN done!
2025-07-28 16:34:13,438 cisTopic     INFO     Subsetting data for XQ (47743 of 295888)
2025-07-28 16:34:25,795 cisTopic     INFO     Computing p-value for XQ
2025-07-28 16:55:09,052 cisTopic     INFO     Computing log2FC for XQ
2025-07-28 16:55:10,391 cisTopic     INFO     XQ done!
2025-07-28 16:55:40,339 cisTopic     INFO     Subsetting data for YB (33091 of 295888)
2025-07-28 16:55:52,805 cisTopic     INFO     Computing p-value for YB
2025-07-28 17:15:55,380 cisTopic     INFO     Computing log2FC for YB
2025-07-28 17:15:56,722 cisTopic     INFO     YB done!


'\nfrom pycisTopic.clust_vis import plot_imputed_features\nplot_imputed_features(\n    cistopic_obj,\n    reduction_name=\'UMAP\',\n    imputed_data=imputed_acc_obj,\n    features=[markers_dict[x].index.tolist()[0] for x in [\'1\',\'2\',\'4\',\'6\']],\n    scale=False,\n    num_columns=4\n)\n\nprint("Number of DARs found:")\nprint("---------------------")\nfor x in markers_dict:\n    print(f"  {x}: {len(markers_dict[x])}")\n'

## SAVE REGION SETS

In [13]:
os.makedirs(os.path.join(out_dir, "region_sets"), exist_ok = True)
os.makedirs(os.path.join(out_dir, "region_sets", "Topics_otsu"), exist_ok = True)
os.makedirs(os.path.join(out_dir, "region_sets", "Topics_top_3k"), exist_ok = True)
os.makedirs(os.path.join(out_dir, "region_sets", "DARs_cell_type"), exist_ok = True)

from pycisTopic.utils import region_names_to_coordinates

for topic in region_bin_topics_otsu:
    region_names_to_coordinates(
        region_bin_topics_otsu[topic].index
    ).sort_values(
        ["Chromosome", "Start", "End"]
    ).to_csv(
        os.path.join(out_dir, "region_sets", "Topics_otsu", f"{topic}.bed"),
        sep = "\t",
        header = False, index = False
    )

for topic in region_bin_topics_top_3k:
    region_names_to_coordinates(
        region_bin_topics_top_3k[topic].index
    ).sort_values(
        ["Chromosome", "Start", "End"]
    ).to_csv(
        os.path.join(out_dir, "region_sets", "Topics_top_3k", f"{topic}.bed"),
        sep = "\t",
        header = False, index = False
    )

for cell_type in markers_dict:
    region_names_to_coordinates(
        markers_dict[cell_type].index
    ).sort_values(
        ["Chromosome", "Start", "End"]
    ).to_csv(
        os.path.join(out_dir, "region_sets", "DARs_cell_type", f"{cell_type}.bed"),
        sep = "\t",
        header = False, index = False
    )

In [14]:
import pickle
pickle.dump(
    cistopic_obj,
    open(os.path.join(out_dir, "cistopic_obj.pkl"), "wb")
)